# Intelligent Agents: Vacuum-cleaner World


## Instructions

Total Points: undergrad 10, graduate students 11

Complete this notebook and submit it. The notebook needs to be a complete project report with 

* your implementation,
* documentation including a short discussion of how your implementation works and your design choices, and
* experimental results (e.g., tables and charts with simulation results) with a short discussion of what they mean. 

Use the provided notebook cells and insert additional code and markdown cells as needed.

## Introduction

In this assignment you will implement a simulator environment for an automatic vacuum cleaner robot, a set of different agent programs, and perform a comparison study for cleaning a single room. Focus on the __cleaning phase__ which starts when the robot is activated and ends when the last dirty square is cleaned. Someone else will take care of the agent program needed to navigate back to the charging station after the room is clean.

## PEAS description of the cleaning phase

__Performance Measure:__ Each action costs 1 energy unit. The performance is measured as the sum of the energy units used to clean the whole room.

__Environment:__ A room with $n \times n$ squares where $n = 5$. Dirt is randomly placed on each square with probability $p = 0.2$. For simplicity, you can assume that the agent knows the size and the layout of the room (i.e., it knows $n$). To starte, the agent is placed on a random square.

__Actuators:__ The agent can `clean` the current square or move to an adjacent square by going `north`, `east`, `south`, or `west`.

__Sensors:__ Four bumper sensors, one for north, east, south, and west; a dirt sensor reporting dirt in the current square.  


## The agent program for a simple randomized agent

The agent program is a function that gets sensor information (the current percepts) as the arguments. The arguments are:

* A dictionary with boolean entries for the for bumper sensors `north`, `east`, `west`, `south`. E.g., if the agent is on the north-west corner, `bumpers` will be `{"north" : True, "east" : False, "south" : False, "west" : True}`.
* The dirt sensor produces a boolean.

The agent returns the chosen action as a string.

Here is an example implementation for the agent program of a simple randomized agent:  

In [522]:
from numpy import random

actions = ["north", "east", "west", "south", "suck"]


def simple_randomized_agent(bumpers, dirty):
    return random.choice(actions)

In [523]:
simple_randomized_agent({"north" : True, "east" : False, "south" : False, "west" : True}, True)

'west'

__Note:__ This is not a rational intelligent agent. It ignores its sensors and may bump into a wall or not clean a dirty square. You will be asked to implement rational agents below.

## Simple environment example

This simple environment is infinite in size (bumpers are always `False`) and every square is always dirty, even if the agent cleans it. The environment function returns the performance measure which is here the number of cleaned squares (since all squares are constantly dirty, it is the number of `suck` actions by the agent). 

In [524]:
import numpy as np
    
arr = np.zeros((5,5), dtype='bool')
test = [[1,2],[3,4]]
ad = 0

for i in range(len(arr)):
    for j in range(len(arr[i])):
        arr[i,j] = np.random.choice([True,False])
        ad = ad + arr[i,j] 
print(arr)
print("Dirty: ", ad)

position = [random.choice(range(5)), random.choice(range(5))]

print(position)
bumpers = {"north" : position[1] == 0, "south" : position[1] == 5 - 1
                  , "west" : position[0] == 0, "east" : position[0] == 5 - 1 }
print(bumpers)
print(arr[1][1])
some_list = [True, False]
if(some_list):
    print("expected")



[[False False  True False  True]
 [ True  True  True False  True]
 [ True  True  True False False]
 [False False False False False]
 [ True False  True False False]]
Dirty:  11
[0, 0]
{'north': True, 'south': False, 'west': True, 'east': False}
True
expected


In [525]:
def simple_environment(agent, max_steps, verbose = True):
    num_cleaned = 0
    
    for i in range(max_steps):
        dirty = True
        bumpers = {"north" : False, "south" : False, "west" : False, "east" : False}

        action = agent(bumpers, dirty)
        if (verbose): print("step", i , "- action:", action) 
        
        if (action == "suck"): 
            num_cleaned = num_cleaned + 1
        
    return num_cleaned
        


Do one simulation run with 20 steps.

In [526]:
simple_environment(simple_randomized_agent, max_steps = 20)

step 0 - action: suck
step 1 - action: east
step 2 - action: east
step 3 - action: south
step 4 - action: east
step 5 - action: north
step 6 - action: north
step 7 - action: south
step 8 - action: east
step 9 - action: east
step 10 - action: east
step 11 - action: east
step 12 - action: south
step 13 - action: south
step 14 - action: east
step 15 - action: suck
step 16 - action: east
step 17 - action: suck
step 18 - action: east
step 19 - action: north


3

# Tasks

_Submission Instructions:_ Use this notebook to prepare your submission. Complete this section with your code and results. You can add additional Markdown blocks for your description, comments in the code and use mathplotlib to produce charts. 

_Note:_ Try to keep the code simple! In this course, we want to learn about the algorithms and we often do not need to use object-oriented design. 


## Task 1: Implement a simulation environment [2 Points]

The simple environment above is not very realistic. Your environment simulator needs to follow the PEAS description from above. It needs to:

* Initialize the environment by storing the state of each square (clean/dirty) and making some dirty.
* Keep track of the agent's position.
* Call the agent function repeatedly and provide the agent function with the sensor inputs.  
* React to the agent's actions. E.g, by removing dirt from a square or moving the agent around unless there is a wall in the way.
* Keep track of the performance measure. That is, track the agent's actions until all dirty squares are clean and count the number of actions it takes the agent to complete the task.

The easiest implementation for the environment is to hold an 2-dimensional array to represent if squares are clean or dirty and to call the agent function in a loop until all squares are clean or a predefined number of steps have been reached (i.e., the robot runs out of energy).

The simulation environment needs to work with the simple randomized agent program from above and then it can be used for your agent implementation in the tasks below.

In [527]:
# Your code and description goes here
import numpy as np # wanted to get a feel for np, using their arrays

def display_environment(env_array, pos, step): # if verbose, this displays the grid and the agent in the env
    print('+------------------------------------------+')
    print("Step:",step)
    print("At position (", pos[1] ,",", pos[0],")", "isDirty:" , env_array[pos[0],pos[1]])
    for i in range(len(env_array)):
        for j in range(len(env_array[i])):
            if pos[0] == i and pos[1] == j and env_array[i,j]:
                print("0  ", end='')
            elif pos[0] == i and pos[1] == j:
                print("O  ", end='')
            elif env_array[i,j]:
                print('#  ', end='')
            else:
                print('_  ', end='')
        print()
        print()

# takes agent, max steps, size of env, and whether you want a description of every step
def environment(agent, max_steps, env_size_n, verbose = False):
    
    #init stuff
    num_dirty = 0
    num_cleaned = 0
    energy_units_used = 0
    
    position = [random.choice(range(env_size_n)), random.choice(range(env_size_n))]
    
    arr = np.zeros((env_size_n, env_size_n), dtype='bool')
    
    #randomize dirt in the grid
    for i in range(len(arr)):
        for j in range(len(arr[i])):
            arr[i,j] = random.choice([True,False, False]) # 1/3 chance of dirty
            num_dirty = num_dirty + arr[i,j]
    
    #agent interaction
    for i in range(max_steps):
        #if agent cleaned all squares, stop agent
        if num_cleaned == num_dirty:
            if verbose:print("Finished cleaning!")
            break

            
     # x is position[1] y is position[0]
    
    #display environment. Set Bumpers with relation to the vacuums position
        if verbose: display_environment(arr, position, i+1)
        #update bumper sensor based on position
        bumpers = {"north" : position[0] == 0, "south" : position[0] == env_size_n - 1
                  , "west" : position[1] == 0, "east" : position[1] == env_size_n - 1 }
        
        # call agent function, pass bumper information and get current spot dirtiness (arr[y,x] gets us the bool)
        action = agent(bumpers, arr[position[0], position[1]])
        energy_units_used += 1
        
        if verbose: print("Action:",action,end='\n\n')
            
        # if sucked only increment counter if the spot was dirty. Clear the spot if it was dirty
        if (action == 'suck'):
            num_cleaned = num_cleaned + arr[position[0], position[1]]
            if(arr[position[0],position[1]]):
                arr[position[0],position[1]] = False
                if verbose: print("Cleaned!")
                    
        elif action == None : # only in the event that the agent does not return anything
            if verbose: print("Did Nothing!")
        # if movement was made check if movement is possible: if not dont do it
        elif(action == "south" or action == "north"):
            position[0] = position[0] - (not bumpers["north"]) if (action == "north")\
            else position[0] + (not bumpers["south"])
        else:
            position[1] = position[1] - (not bumpers["west"]) if (action == "west")\
            else position[1] + (not bumpers["east"])
        
    return (num_cleaned,num_dirty,energy_units_used) # we return a tuple of important data for future comparison

In [528]:
environment(simple_randomized_agent, 10, 5,0)

(1, 10, 10)

## Task 2:  Implement a simple reflex agent [1 Point] 

The simple reflex agent randomly walks around but reacts to the bumper sensor by not bumping into the wall and to dirt with sucking.

In [529]:
# Your code and description goes here
def simple_reflex_agent(bumpers, dirty):
    if dirty : return 'suck' # if dirty we prioritize cleaning
    else:
        actions = ["north", "east", "west", "south"] # list of actions
        action_to_take = np.random.choice(actions)   # action will be random
        while bumpers[action_to_take]: # Take an action that doesn't hit the wall
            action_to_take = np.random.choice(actions)
        return action_to_take

In [530]:
environment(simple_reflex_agent, 10, 5,0)

(2, 11, 10)

## Task 3: Implement a model-based reflex agent [3 Point]

This agent keeps track of the location and remembers where it has cleaned. Assume the agent knows how many squares the room has. It can move to a corner to determin its location and then is able to use more advanced navigation.

_Note on implementing the state:_ You can use a global variable. In Python, you have to use the keyword `global` in your function for this to work (see: https://www.programiz.com/python-programming/global-keyword). Alternatively, you can define a class for your agent with a member variable for the state and a function for the agent program (see: https://www.w3schools.com/python/python_classes.asp). 

Describe how you define the __agent state__ and how your agent works before implementing it.

In [542]:
# Your short description goes here
# I will be using a class.
# The instance of the agent will have a couple of variables to keep track of the state.
# The first thing to consider is whether the agent is 'oriented' - if it knows where it is in the room
# My agent will begin by navigating to a corner. It will keep track of its movements with respect to the
# starting point. It will then calculate its starting point once it's reached a corner since this should be a 
# N x N Room.
# from there, It will begin Moving through the room by alternating north/south across the room.
# it will do this until it runs out of enery units or until the room is completely cleaned.

In [532]:
# Your code goes here
class model_based_agent:
  
    def __init__(self, n_room_size):
        
        self.size = n_room_size
        
        self.starting_pos = [0, 0]       # initially keeps track of movements relative to start
        self.current_route = None        # keeps track of the current direction agent is moving
        self.cur_pos = [False, False]    # keeps track of position of the agent      
        self.oriented = [False,False]    # lets the agent know if it is oriented or not      
        self.dir_orient = [False, False] # keeps track of the directions for orienting the agent  
        
    # function to orient the agent into a corner
    def orient_agent(self, bumpers):
        if(self.oriented[0] and self.oriented[1]):
            self.dir_orient[0] = "north" if bumpers['north'] else 'south'
            self.dir_orient[1] = 'east' if bumpers['east'] else 'west'
            return 'south' if bumpers['north'] else 'north'
        elif not (self.oriented[0]):
            # go either north or south if not decided
            if not self.dir_orient[0]: 
                self.dir_orient[0] = np.random.choice(["north", "south"])
            self.starting_pos[0] += self.update_pos(self.starting_pos[0], self.dir_orient[0], 1)
            return self.dir_orient[0]
            
        elif not self.oriented[1]:
            # either east or west if not decided
            if (not self.dir_orient[1]): 
                self.dir_orient[1] = np.random.choice(['east', 'west'])
            self.starting_pos[1] += self.update_pos(self.starting_pos[1], self.dir_orient[1], 1)
            return self.dir_orient[1]
        else:
            return self.clean_room(bumpers)
   
    # function to update the agent's position tracker depending on the move.
    def update_pos(self,pos, mov_dir, negative = False):
        # fix
        mov_val = -1 if mov_dir == 'north' or mov_dir == 'west'\
         else 1
        return mov_val if not negative else 0 - mov_val
        
    # if bumpers detected, we are at an edge: either move to the next column or down/up the column
    def move_at_edge(self, bumpers):
        if (self.current_route == 'west' or self.current_route == 'east'):
            self.current_route = 'north' if bumpers['south'] else 'south'
            self.cur_pos[0] += self.update_pos(self.cur_pos[0], self.current_route)
        else:
            self.current_route = 'east' if self.dir_orient[1] == 'west' else 'west'
            self.cur_pos[1] += self.update_pos(self.cur_pos[1], self.current_route)
        return self.current_route
            
        
    def clean_room(self, bumpers):
        
        # if current route not set, then we have just oriented the agent.
        # we need to figure out where we are and what direction to move.
        if self.current_route == None:
            x = self.size - 1 if bumpers['east']  else 0
            y = self.size -1 if bumpers['south']  else 0
            self.cur_pos = [y, x]
            self.starting_pos = [y + self.starting_pos[0], x + self.starting_pos[1]]
            self.current_route = 'south' if self.dir_orient[0] == 'north' else 'north'
            return self.current_route
        elif bumpers['south'] or bumpers['north']: # reach an edge.
            return self.move_at_edge(bumpers)
        else:
            return self.current_route # if in the middle, keep going until at an edge.
                                                                            
                                                  
    def model_based_agent_function(self, bumpers, dirty):
        if dirty : return 'suck'        # prioritze cleaning
        elif( not (self.oriented[0] and  self.oriented[1])):  # next prioritze orienting.
            # get bumper data
            self.oriented[0] = bumpers["north"] or bumpers["south"]
            self.oriented[1] = bumpers["east"] or bumpers["west"]
            return self.orient_agent(bumpers)
        else:
            return self.clean_room(bumpers) # next clean room
        
        
        

In [533]:
agent = model_based_agent(10)
model_based_agent_function = agent.model_based_agent_function
environment(model_based_agent_function, 51, 5,0)

(13, 13, 39)

## Task 4: Simulation study [3 Points]

Compare the performance of the agents using different size environments. E.g., $5 \times 5$, $10 \times 10$ and
$100 \times 100$. Use at least 100 random runs for each. Present the results in a suitable format (tables, graphs) and discuss the differences.

In [534]:
# Data format
#  N | Amount_cleaned | amount_dirty | steps_used | max_steps

## Gathering data

In the Following few cells, I gather the data for each agent to anaylze how it performs in N-sized square rooms. The environment function returns how many squares cleaned, how many dirty squares were generated, as well as how many energy units were used. This data is thrown into a list and is used to make a pandas dataframe, which is displayed below the python code This should be enough to determine how effective the agents perform with relation to one another

In [535]:
# This is is for the randomized agent
import pandas as pd

data_simple_agent = []
N_range = [5, 10, 100]
for i in N_range:
    max_steps = 2*i*i+int(i/3)
    for j in range(100):
        #performance p is the returned data from environment
        p = environment(simple_randomized_agent,max_steps,i)
        data_simple_agent.append([i,p[0],p[1],p[2],max_steps])
simple_df = pd.DataFrame(data_simple_agent, \
            columns = ['N', 'Cleaned', 'Dirty', 'Steps', 'Max Steps'])

simple_df

,N,Cleaned,Dirty,Steps,Max Steps
0,5,1,10,51,51
1,5,1,7,51,51
2,5,2,6,51,51
3,5,3,7,51,51
4,5,1,10,51,51
...,...,...,...,...,...
295,100,616,3276,20033,20033
296,100,583,3323,20033,20033
297,100,577,3298,20033,20033
298,100,657,3375,20033,20033


In [536]:
# this is for the reflex agent
data_reflex_agent = []
for i in N_range:
    max_steps = 2*i*i+int(i/3)
    for j in range(100):
        #performance p is the returned data from environment
        p = environment(simple_reflex_agent,max_steps,i)
        data_reflex_agent.append([i,p[0],p[1],p[2],max_steps])
reflex_df = pd.DataFrame(data_reflex_agent, \
            columns = ['N', 'Cleaned', 'Dirty', 'Steps', 'Max Steps'])

reflex_df

,N,Cleaned,Dirty,Steps,Max Steps
0,5,7,9,51,51
1,5,4,8,51,51
2,5,7,10,51,51
3,5,4,8,51,51
4,5,4,5,51,51
...,...,...,...,...,...
295,100,1484,3267,20033,20033
296,100,1371,3305,20033,20033
297,100,1296,3316,20033,20033
298,100,1352,3348,20033,20033


In [537]:
# This is for the model agent
data_model_agent = []
for i in N_range:
    max_steps = 2*i*i+int(i/3)
    for j in range(100):
        agent = model_based_agent(i)
        model_agent_function = agent.model_based_agent_function
        #performance p is the returned data from environment
        p = environment(model_agent_function,max_steps,i)
        data_model_agent.append([i,p[0],p[1],p[2],max_steps])
model_df = pd.DataFrame(data_model_agent, \
            columns = ['N', 'Cleaned', 'Dirty', 'Steps', 'Max Steps'])

model_df

,N,Cleaned,Dirty,Steps,Max Steps
0,5,7,7,30,51
1,5,10,10,36,51
2,5,11,11,36,51
3,5,7,7,32,51
4,5,11,11,34,51
...,...,...,...,...,...
295,100,3292,3292,13312,20033
296,100,3283,3283,13355,20033
297,100,3292,3292,13399,20033
298,100,3309,3309,13354,20033


# Averages / Performance

Here's how I got to the tables below: first we calculate the percentage cleaned as well as the percentage of the energy used for the robot. Once we got those, we took the averages and that is what is displayed in the following tables.

In [538]:
simple_df['%Cleaned'] = simple_df['Cleaned'] / simple_df['Dirty']
reflex_df['%Cleaned'] = reflex_df['Cleaned'] / reflex_df['Dirty']
model_df['%Cleaned']  = model_df['Cleaned'] / model_df['Dirty']

simple_df['%Energy Used'] = simple_df['Steps'] / simple_df['Max Steps']
reflex_df['%Energy Used'] = reflex_df['Steps'] / reflex_df['Max Steps']
model_df['%Energy Used']  = model_df['Steps'] / model_df['Max Steps']

## Simple Randomized

In [539]:
simple_df_avg = simple_df.groupby(['N']).mean()
simple_df_avg[['%Cleaned', '%Energy Used']]

,%Cleaned,%Energy Used
N,,
5,0.265136,1.0
10,0.225617,1.0
100,0.184606,1.0


## Simple Reflex

In [540]:
reflex_df_avg = reflex_df.groupby(['N']).mean()
reflex_df_avg[['%Cleaned', '%Energy Used']]

,%Cleaned,%Energy Used
N,,
5,0.657039,0.983922
10,0.571022,1.000000
100,0.382303,1.000000


## Model Based

In [541]:
model_df_avg = model_df.groupby(['N']).mean()
model_df_avg[['%Cleaned', '%Energy Used']]

,%Cleaned,%Energy Used
N,,
5,1.0,0.646078
10,1.0,0.676650
100,1.0,0.670314


## Graphs
### Cleaning Performance
![](Cleaning.png)
### Energy Usage
** This is For battery usage. Typo in the Graph title
![](BatterUse.png)

## Discussion of simulation

I think it is fair to say that the model-based agent in this N x N square room scenario was the better performer considering the two main performance measures (cleaning and energy units used). I think (at least visually), the graphs clearly tell that story as well. If we pay attention to just the cleaning performance, we can easily tell that the reflex agent performs better than the randomized agent, and the model-based agent performs better than either of the two. We can see that the randomized and reflex agents both suffer as the N increases, while the model-based agent is able to fully clean the room under the same circumstances.

The Energy usage also shows that the Model-based agent was the best of the three as well. Notice that for the max-steps, I gave the formula: 

$2 N^2 + {\frac{1}{3}}$

Where $N$ is the size of the $N {\times} N$ Room

The reason for this was to give the agents enough moves, even if it started at an un-optimal position, and enough to go around and suck the dirt. I will admit that this was a little biased towards the un-optimal position of the Model-based agent, however, it should have been more than enough in all scenarios for every agent, and yet two of the agents were still unable to succeed in cleaning the rooms in most cases. The Model-based agent was the only agent that was able to use the energy somewhat-resourcefully ( though only in this easy example), since it did not always start at an un-optimal position. Perhaps a future implementation can make the agent more efficient.

I believe the reason for this landslide victory for the model-based agent is that the model-based agent does not act completely on randomness. The Reflex agent and Randomized agent, though only one carries the name 'randomized', are both essentially randomized; the only difference being that the reflex agent responds to its percepts in a limited fashion. The Model-based agent has structure in its movements and executes identically in every scenario. This is great for this N x N example, but it would be completely destroyed  if any other little difference were to be added.

## Task 5: Robustness of the agent implementations [1 Point] 

Describe how your agent implementations will perform 

* if it is put into a rectangular room with unknown size, 
* if the cleaning area can have an iregular shape (e.g., a hallway connecting two rooms), or 
* if the room contains obstacles (i.e., squares that it cannot pass through and trigger the bumper sensors).


### Model based agent:
I think as it stands, my model-based implementation would be okay in a rectangular room. One possible issue is that the logic for being aware of its locaion needs to know the size of the room, and if we throw in variable x,y dimensions, as it stands, it would be unable to do this accurately. In fact I believe there would probably be some out of bound errors since the agent assumes it is in a square room. Once we expand past a square grid, the agent will go out of bounds. However, the agent would still be able to clean most or a lot of the squares in the room. The reason for this is that, as I have implemented it, it will still be able to find a corner and keep going up and down across the room.

Same goes for irregular shapes: there would need to be additional logic to handle differences in dimensions, or I would need to refactor the entire logic to be able to work in any type of shape. The agent as it stands would still try to find a corner and navigate up and down across the room. However, because of the irregular shape, the agent would assume it has hit a north/south wall in a square room, rather than an irregular-shaped room

Unfortunately, obstacles would also throw off my poor little agent. As it stands an obstacle would confuse it into thinking it has reached a wall, and it would go to the next column to clean. This would mean it would be unable to clean all squares past that obstacle, and it would be confused into thinking it was in a smaller sized square.

### Randomized and Reflex agents

As for the other agents, they would perform just the same, regardless of the environment. In fact they may perform better than the model-based agent because they would continue to move randomly, and their performance in square rooms would be just about the same in rectangle rooms, irregular rooms, and rooms with obstacles. The Model-based agent logic would be thrown out the door, while the logic for these two agents would continue to perform exactly the same in any given scenario.

## Graduate student advanced task: Obstacles [1 Point]

__Undergraduate students:__ This is a bonus task you can attempt if you like [+1 Bonus point].

1. Change your simulation environment tor run experiments for the following problem: Add random obstacle squares that also trigger the bumper sensor. The agent does not know where the obstacles are. Observe how this changes the performance of the three implementations.

2. Describe what would need to be done to perform better with obstacles. Add code if you can. 

In [12]:
# Your code and discussion goes here

## More advanced tasks to think about

You can think about these:

* __Unknown environment with obstacles:__ Implement an agent for an environment where the agent does not know how large the environment is (we assume it is rectangular), where it starts or where the obstacles are. An option would be to always move to the closest unchecked/uncleaned square.

* __Utility-based agent:__ Change the environment, so each square has a fixed probability of getting dirty again. We assume the agent has learned this information over time. For the implementation, we give this information to the agent as a 2-dimensional array of probabilities  Cleaning one dirty square produces a utility of 1. Implement a utility-based agent that maximizes the expected utility over one full charge which lasts for 10000 time steps. This is very tricky!